<a href="https://colab.research.google.com/github/randywreed/sav2csv/blob/master/variable_sav_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install voila
!jupyter serverextension enable voila —sys-prefix


     |████████████████████████████████| 6.9MB 2.1MB/s 
     |████████████████████████████████| 552kB 41.4MB/s 
     |████████████████████████████████| 215kB 18.4MB/s 
     |████████████████████████████████| 112kB 37.0MB/s 
  Found existing installation: nbconvert 5.6.1
    Uninstalling nbconvert-5.6.1:
      Successfully uninstalled nbconvert-5.6.1
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5


Enabling: voila
- Writing config: /root/.jupyter
    - Validating...
Error loading server extension voila
      X is voila importable?
Enabling: —sys-prefix
- Writing config: /root/.jupyter
    - Validating...
Error loading server extension —sys-prefix
      X is —sys-prefix importable?


In [2]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
!pip install pyreadstat
import pandas as pd

     |████████████████████████████████| 2.0MB 2.9MB/s 


In [4]:
import ipywidgets as widgets

In [5]:
from IPython.display import display

In [6]:
upload=widgets.FileUpload(accept=".sav",multiple=False)
upload

FileUpload(value={}, accept='.sav', description='Upload')

In [7]:
with open (upload.metadata[0]['name'],"w+b") as i:
  i.write(upload.data[0])

In [8]:
import pyreadstat
df,meta=pyreadstat.read_sav(upload.metadata[0]['name'])

In [9]:
df.colmnns=meta.column_names

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [10]:
klab={}
klab=meta.column_names_to_labels
#type(klab)

In [11]:
labs=[]
for k,v in klab.items():
  #print("{}:{}".format(k,v))
  labs.append([k,v])

In [12]:
df_enriched=pyreadstat.set_value_labels(df,meta,formats_as_category=True)

In [13]:
def strip_it(initialsent):
  import string, re
  en_stops=set(stopwords.words('english'))
  #update stopwords
  en_stops.remove('not')
  newstopwords=['what','why','who','when','where','how','many']
  en_stops.update(newstopwords)
  #remove puntuation, stopwords, and add spaces to runon words 
  translator = str.maketrans('', '', string.punctuation)
  newvar=initialsent.translate(translator)
  #print('after punctuation strip=',newvar)
  newvar=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', newvar)
  #print('add spaces if necessary=',newvar)
  c_split=newvar.split(" ")
  c_strip=""
  c_stripns=""
  for word in c_split:
    if str.lower(word) not in en_stops:
      c_strip=c_strip+word+" "
      c_stripns=c_stripns+word+"_"
  c_stripns=c_stripns.rstrip('_')
  return (c_stripns,c_strip)


In [14]:
def create_options(lab,val):
  out=[]
  ret=strip_it(val)
  out=[lab,ret[0],ret[1],"other"]
  return out

In [15]:
import os
base=os.path.basename(upload.metadata[0]['name'])
newfile=os.path.splitext(base)
#print(newfile[0])
df_enriched.to_csv(newfile[0]+".csv")

In [16]:
if 'google.colab' in str(get_ipython()):
  from google.colab import files
  files.download(newfile[0]+'.csv')
else:
  csv_download_link(df,newfile[0]+'.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>